In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from utils.data_preprocess import * # data_split, DataLabeling, train_val_test_split
from models import *

In [2]:
WINDOW_SIZE = 24

In [3]:
df = pd.read_csv('data\ETHBTC-5m-data.csv')
raw_data = DataLabeling(df, WINDOW_SIZE)
train_df, val_df, test_df = train_val_test_split(raw_data.labelled_data)
Data = DataPreprocess(train_df, val_df, test_df, WINDOW_SIZE, label_size=1, label_columns=['Trend'], shift=0)

In [10]:
model = CDT_1D_model(24, 5)

In [11]:
test_model = model.model

In [12]:
test_model.fit(Data.train,
               epochs=20000,
               validation_data=Data.val,
               steps_per_epoch=len(Data.train),
               validation_steps=int(0.15*len(Data.val)),
               callbacks=[create_model_checkpoint(model_name=test_model.name, save_path='without_TIs'),
                          tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                          tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, min_lr=0.00001, factor=0.5)])

Epoch 1/20000
11867/11867 [==============================] - 165s 13ms/step - loss: 0.7845 - accuracy: 0.5678 - val_loss: 482.4799 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/20000
11867/11867 [==============================] - 140s 12ms/step - loss: 0.7720 - accuracy: 0.5806 - val_loss: 0.0303 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/20000
11867/11867 [==============================] - 132s 11ms/step - loss: 0.7569 - accuracy: 0.5905 - val_loss: 203.6187 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/20000
11867/11867 [==============================] - 133s 11ms/step - loss: 0.7480 - accuracy: 0.5930 - val_loss: 83.7447 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 5/20000
11867/11867 [==============================] - 136s 11ms/step - loss: 0.7423 - accuracy: 0.5951 - val_loss: 0.0177 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/20000
11867/11867 [==============================] - 138s 12ms/step - loss: 0.7399 - accuracy: 0.5963 - val_loss: 0.0168 - val_accuracy: 1.0000 - lr: 

KeyboardInterrupt: 

In [21]:
test_model = tf.keras.models.load_model('data_without_TI\CDT-1D_model')
test_model.evaluate(Data.train)

11867/11867 [==============================] - 57s 5ms/step - loss: 2.5824 - accuracy: 0.4772


[2.5823657512664795, 0.4772425591945648]

In [18]:
tf.argmax(test_model.predict(Data.train), axis=1)

<tf.Tensor: shape=(142393,), dtype=int64, numpy=array([2, 2, 2, ..., 2, 2, 2], dtype=int64)>

In [23]:
val_df.Trend.value_counts()

2    104
0     87
1      1
Name: Trend, dtype: int64